In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
np.random.seed(1)

In [3]:
# Ejericicio 1.1


def simula_unif(N, dim, rango):
	return np.random.uniform(rango[0],rango[1],(N,dim))

def simula_gaus(N, dim, sigma):
    media = 0    
    out = np.zeros((N,dim),np.float64)        
    for i in range(N):
        # Para cada columna dim se emplea un sigma determinado. Es decir, para 
        # la primera columna (eje X) se usará una N(0,sqrt(sigma[0])) 
        # y para la segunda (eje Y) N(0,sqrt(sigma[1]))
        out[i,:] = np.random.normal(loc=media, scale=np.sqrt(sigma), size=dim)
    
    return out


def simula_recta(intervalo):
    points = np.random.uniform(intervalo[0], intervalo[1], size=(2, 2))
    x1 = points[0,0]
    x2 = points[1,0]
    y1 = points[0,1]
    y2 = points[1,1]
    # y = a*x + b
    a = (y2-y1)/(x2-x1) # Calculo de la pendiente.
    b = y1 - a*x1       # Calculo del termino independiente.
    
    return a, b

In [4]:
###############################################################################
# Ejercicio 1.2

# La funcion np.sign(0) da 0, lo que nos puede dar problemas
def signo(x):
	if x >= 0:
		return 1
	return -1

def f(x, y, a, b):
	return signo(y - a*x - b)

In [5]:
def error_rate(x, y, a, b):
    """
    Funcion para calcular los ratios de acierto y error entre los valores
    reales de las etiquetas y los predichos.
    
    :param x: Array de vectores de caracteristicas
    :param y: Array de etiquetas
    :param a: Pendiente de la recta
    :param b: Valor independiente de la recta
    
    :return Devuelve el ratio de aciertos y el ratio de errores
    """
    
    # Crear lista de y predichas
    predicted_y = []
    
    # Predecir cada valor
    for value in x:
        predicted_y.append(f(value[0], value[1], a, b))
    
    # Convertir a array
    predicted_y = np.array(predicted_y)
    
    return np.mean(y == predicted_y), np.mean(y != predicted_y)

In [6]:
# Simular 50 puntos uniformemente distribuidos en el cuadrado [-50, 50] x [-50, 50]
x = simula_unif(50, 2, [-50, 50])

# Simular una recta en el rango [-50, 50] y calcular sus coeficientes
a, b = simula_recta([-50, 50])

# Obtener los valores de la etiqueta
# Guardarlos primero en una lista y convertirlos en un array
y = []

for value in x:
    y.append(f(value[0], value[1], a, b))

y = np.array(y)

In [7]:
# Obtener ratios de acierto y error
accuracy, error = error_rate(x, y, a, b)

print('Ratio de aciertos: {}'.format(accuracy))
print('Ratio de error: {}'.format(error))

Ratio de aciertos: 1.0
Ratio de error: 0.0


In [8]:
def insert_noise(y, ratio=0.1):
    """
    Función para insertar ruido en una muestra de forma proporcional en cada
    clase.
    
    :param y: Etiquetas sobre las que insertar ruido
    :param ratio: Ratio de elementos de cada clase que modificar
    """
    # Obtener número de elementos sobre los que aplicar ruido
    # (redondear)
    noisy_pos = round(np.where(y == 1)[0].shape[0] * ratio)
    noisy_neg = round(np.where(y == -1)[0].shape[0] * ratio)
    
    # Obtener las posiciones de forma aleatoria
    pos_index = np.random.choice(np.where(y == 1)[0], noisy_pos, replace=False)
    neg_index = np.random.choice(np.where(y == -1)[0], noisy_neg, replace=False)
    
    # Cambiar los valores
    y[pos_index] = -1
    y[neg_index] = 1

In [9]:
y_noisy = np.copy(y)

insert_noise(y_noisy)

In [10]:
accuracy, error = error_rate(x, y_noisy, a, b)

print('Ratio de aciertos: {}'.format(accuracy))
print('Ratio de error: {}'.format(error))

Ratio de aciertos: 0.9
Ratio de error: 0.1


In [11]:
# Funcion proporcionada para mostrar las graficas
def plot_datos_cuad(X, y, fz, title='Point cloud plot', xaxis='x axis', yaxis='y axis'):
    #Preparar datos
    min_xy = X.min(axis=0)
    max_xy = X.max(axis=0)
    border_xy = (max_xy-min_xy)*0.01
    
    #Generar grid de predicciones
    xx, yy = np.mgrid[min_xy[0]-border_xy[0]:max_xy[0]+border_xy[0]+0.001:border_xy[0], 
                      min_xy[1]-border_xy[1]:max_xy[1]+border_xy[1]+0.001:border_xy[1]]
    grid = np.c_[xx.ravel(), yy.ravel(), np.ones_like(xx).ravel()]
    pred_y = fz(grid)
    # pred_y[(pred_y>-1) & (pred_y<1)]
    pred_y = np.clip(pred_y, -1, 1).reshape(xx.shape)
    
    #Plot
    f, ax = plt.subplots(figsize=(8, 6))
    contour = ax.contourf(xx, yy, pred_y, 50, cmap='RdBu',vmin=-1, vmax=1)
    ax_c = f.colorbar(contour)
    ax_c.set_label('$f(x, y)$')
    ax_c.set_ticks([-1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1])
    ax.scatter(X[:, 0], X[:, 1], c=y, s=50, linewidth=2, 
                cmap="RdYlBu", edgecolor='white')
    
    XX, YY = np.meshgrid(np.linspace(round(min(min_xy)), round(max(max_xy)),X.shape[0]),
                         np.linspace(round(min(min_xy)), round(max(max_xy)),X.shape[0]))
    positions = np.vstack([XX.ravel(), YY.ravel()])
    ax.contour(XX,YY,fz(positions.T).reshape(X.shape[0],X.shape[0]),[0], colors='black')
    
    ax.set(
       xlim=(min_xy[0]-border_xy[0], max_xy[0]+border_xy[0]), 
       ylim=(min_xy[1]-border_xy[1], max_xy[1]+border_xy[1]),
       xlabel=xaxis, ylabel=yaxis)
    plt.title(title)
    plt.show()

In [12]:
# Función del primer apartado
def f1(X):
    return (X[:, 0] - 10) ** 2 + (X[:, 1] - 20) ** 2 - 400

# Función del segundo apartado
def f2(X):
    return 0.5 * (X[:, 0] + 10) ** 2 + (X[:, 1] - 20) ** 2 - 400

# Función del tercer apartado
def f3(X):
    return 0.5 * (X[:, 0] - 10) ** 2 - (X[:, 1] + 20) ** 2 - 400

# Función del cuarto apartado
def f4(X):
    return X[:, 1] - 20 * X[:, 0] ** 2  - 5 * X[:, 0]  + 3

In [13]:
def error_rate_func(x, y, func):
    """
    Función para calcular los ratios de acierto y error al predecir un conjunto
    de puntos x mediante una función func, con respecto de los valores reales y
    
    :param x: Puntos que se usarán para predecir
    :param y: Etiquetas reales de los puntos
    :param func: Función con la que se predecirán los puntos
    
    :return Devuelve el ratio de puntos predichos correctamente y el ratio de
            puntos predichos erróneamente
    """
    
    # Predecir las etiquetas
    predicted_y = func(x)
    
    # Hacer que los valores predichos estén en el rango (-1, 1)
    predicted_y = np.clip(predicted_y, -1, 1)
    
    return np.mean(predicted_y == y), np.mean(predicted_y != y) 

In [14]:
def adjust_PLA(data, label, max_iter, initial_values):
    """
    Implementación del PLA para ajustar una serie de pesos
    para un perceptrón
    
    :param data: Conjunto de datos con los que entrenar el perceptrón
    :param label: Conjunto de etiquetas, una por cada grupo de características
    :param max_iter: Número máximo de iteraciones (épocas) que hace el PLA
                     (limitar el número de iteraciones en caso de que no converja)
    :param initial_values: Valores iniciales de w
    
    :return Devuelve los pesos obtenidos (w) junto con el número de épocas
            que ha tardado en converger (epoch)
    """
    
    # Copiar valores iniciales de w
    w = np.copy(initial_values)
    
    # Inicializar la convergencia a falso
    convergence = False
    
    # Inicializar el número de épocas realizadas a 0
    epoch = 0
    
    # Mientras no se haya convergido, ajustar el Perceptron
    while not convergence:
        # Incrementar el número de épocas y decir que se ha convergido
        convergence = True
        epoch += 1
        
        # Recorrer cada elemento de los datos con su correspondiente etiqueta
        # Si se ve que el valor predicho no se corresponde con el real
        # se dice que no se ha convergido en esta época
        for x, y in zip(data, label):
            # Calculor valor predicho (función signo)
            predicted_y = signo(w.dot(x.reshape(-1, 1)))
            
            # Comprobar si el valor predicho es igual al real
            if predicted_y != y:
                w += y * x
                convergence = False  
        
        # Si se ha alcanzado el máximo de épocas, terminar
        if epoch == max_iter:
            break            
    
    return w, epoch

In [15]:
# Crear el conjunto de datos añadiendo una columna con unos a los x
data = np.c_[np.ones((x.shape[0], 1), dtype=np.float64), x]

# Crear array de zeros
zeros = np.array([0.0, 0.0, 0.0])

In [16]:
print('Algoritmo PLA con w_0 = [0.0, 0.0, 0.0]\n')

# Lanzar el algoritmo PLA con w = [0, 0, 0] y guardar la información
w, iter = adjust_PLA(data, y, 10000, zeros)
print('Valor w: {} \tNum. iteraciones: {}'.format(w, iter)) 

Algoritmo PLA con w_0 = [0.0, 0.0, 0.0]

Valor w: [ 3.         24.77725903 68.99701013] 	Num. iteraciones: 4


In [17]:
# Fijamos la semilla
# Se cambia la semilla porque con la primera se obtiene una recta muy mala
np.random.seed(2)

# Simular 100 puntos 2D de forma uniforme en el rango [0, 2]
x_train = simula_unif(100, 2, [0.0, 2.0])

# Simular una recta en el rango [0, 2] y calcular sus coeficientes
a, b = simula_recta([0.0, 2.0])

# Inicializar las etiquetas a una nueva lista
y_train = []

# Recorrer los valores de x_train y generar los valores de las etiquetas
# utilizando la recta de clasificación
for value in x_train:
    y_train.append(f(value[0], value[1], a, b))

# Convertir la lista de etiquetas a array
y_train = np.array(y_train)

In [18]:
def sgdRL(data, labels, initial_w, threshold=0.01, lr=0.01):
    """
    Función que calcula unos pesos para la Regresión Logística mediante
    el Gradiente Descendente Estocástico
    
    :param data: Conjunto de datos
    :param labels: Conjunto de etiquetas
    :param initial_w: Valores iniciales de w
    :param threshold: Límite de las diferencias entre los w de dos épocas con
                      el que parar
    :param lr: Ratio de aprendizaje
    
    :return Devuelve un vector de pesos (w)
    """
    # Copiar el w inicial, los datos y las etiquetas
    w = np.copy(initial_w)
    x_data = np.copy(data)
    y_data = np.copy(labels)
    
    # Obtener número de elementos
    N = x_data.shape[0]
    
    # Establecer una diferencia inicial entre w_(t-1) y w_t
    delta = np.inf
    
    # Mientras la diferencia sea superior al umbral,
    # generar una nueva época e iterar sobre los datos
    while delta > threshold:
        # Crear una nueva permutación y aplicarla a los datos
        # para generar una nueva época
        indexes = np.random.permutation(N)
        x_data = x_data[indexes, :]
        y_data = y_data[indexes]
        
        # Guardar w_(t-1)
        prev_w = np.copy(w)
        
        # Actualizar w con la nueva época
        for x, y in zip(x_data, y_data):
            w = w - lr * gradient_sigmoid(x, y, w)
        
        # Comprobar el nuevo delta
        delta = np.linalg.norm(prev_w - w)
    
    return w.reshape(-1,)

In [19]:
def gradient_sigmoid(x, y, w):
    return -(y * x)/(1 + np.exp(y * w.dot(x.reshape(-1,))))

In [20]:
def error_func(data, labels, w):
    """
    Función para calcular el error en un conjunto de datos.
    
    :param data: Conjunto de datos
    :param labels: Conjunto de etiquetas
    :param w: Vector de pesos
    
    :return Devuelve el error
    """
    
    # Obtener número de elementos e inicializar error inicial
    N = data.shape[0]
    error = 0.0
    
    # Recorrer cada elemento e ir incrementando el error con
    # la función del ERM
    for x, y in zip(data, labels):
        error += np.log(1 + np.exp(-y * w.dot(x.reshape(-1, 1))))
        
    return error[0] / N